<a href="https://colab.research.google.com/github/renzungo/Clarin_Covers_Sent_Analysis/blob/sentiment/04_spacy_ner_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive; drive.mount('/content/drive', force_remount=True)

# Uninstall conflicting libraries
!pip uninstall -y opencv-python opencv-contrib-python opencv-python-headless numpy

# Install required libraries with specified versions
!pip -q install spacy==3.7.5 pandas tqdm numpy==1.26.4
import sys, subprocess, os
import spacy, pandas as pd
from pathlib import Path
from collections import Counter
from tqdm.auto import tqdm
import json # Import the json library

# Ensure model
try:
    spacy.load("es_core_news_md")
except OSError:
    subprocess.run([sys.executable, "-m", "spacy", "download", "es_core_news_md"], check=True)
nlp = spacy.load("es_core_news_md")

WORK_DIR = "/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_pipeline_work"
OUT_DIR = "/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_analytics_out"
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

df_base = pd.read_parquet(os.path.join(WORK_DIR, "base.parquet"))

count_rows = []
span_rows = []

for r in tqdm(df_base.itertuples(index=False), total=len(df_base)):
    text = r.text or ""
    doc = nlp(text)
    people, orgs, places = Counter(), Counter(), Counter()
    for ent in doc.ents:
        if ent.label_ in {"PER","ORG","LOC","GPE"}:
            key = ent.text.strip()
            if ent.label_=="PER": people[key]+=1
            elif ent.label_=="ORG": orgs[key]+=1
            else: places[key]+=1
            # store spans for next step
            span_rows.append({
                "file": r.file, "date": r.date,
                "entity_text": ent.text, "entity_type": ent.label_,
                "start": ent.start_char, "end": ent.end_char
            })

    count_rows.append({
        "file": r.file, "date": r.date,
        "people_json": people.most_common(50),
        "orgs_json": orgs.most_common(50),
        "places_json": places.most_common(50),
    })

# Save counts (as JSON strings) and spans
df_counts = pd.DataFrame(count_rows)
for col in ["people_json","orgs_json","places_json"]:
    # Use json.dumps instead of pd.io.json.dumps
    df_counts[col] = df_counts[col].apply(lambda x: json.dumps(dict(x), ensure_ascii=False))
df_counts.to_parquet(os.path.join(WORK_DIR, "ner_counts.parquet"), index=False)

df_spans = pd.DataFrame(span_rows, columns=["file","date","entity_text","entity_type","start","end"])
df_spans.to_parquet(os.path.join(WORK_DIR, "ner_spans.parquet"), index=False)

print("Wrote:")
print(os.path.join(WORK_DIR, "ner_counts.parquet"))
print(os.path.join(WORK_DIR, "ner_spans.parquet"))

Mounted at /content/drive
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.


  0%|          | 0/652 [00:00<?, ?it/s]

Wrote:
/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_pipeline_work/ner_counts.parquet
/content/drive/MyDrive/Data Justicialista/Clarin Cover Sentiment Analysis/odc_pipeline_work/ner_spans.parquet
